---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [3]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [10]:
def answer_one():
    
    return spam_data['target'].mean()*100

In [11]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    
    return max(vect.get_feature_names(), key=len)

In [18]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    nb = MultinomialNB(alpha=0.1)
    nb.fit(X_train_vectorized, y_train)
    
    predictions = nb.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [22]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():

    vect = TfidfVectorizer().fit(X_train)
    feature_names = np.array(vect.get_feature_names())
    X_train_vectorized = vect.transform(X_train)
    tfidf = X_train_vectorized.max(0).toarray()[0]

    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    small = sorted_tfidf_index[:20]
    large = sorted_tfidf_index[:-21:-1]

    sdf = pd.DataFrame({'FeatureNames': list(feature_names[small]), 'Tfidf': list(tfidf[small])})
    ldf = pd.DataFrame({'FeatureNames': list(feature_names[large]), 'Tfidf': list(tfidf[large])})

    sdf.sort_values(by=['Tfidf', 'FeatureNames'], ascending=[True, True], inplace=True)
    ldf.sort_values(by=['Tfidf', 'FeatureNames'], ascending=[False, True], inplace=True)

    smallest = pd.Series(list(sdf['Tfidf']), index=list(sdf['FeatureNames']))
    largest = pd.Series(list(ldf['Tfidf']), index=list(ldf['FeatureNames']))
    
    return (smallest, largest)

In [86]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [87]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    nb = MultinomialNB(alpha=0.1)
    nb.fit(X_train_vectorized, y_train)
    
    predictions = nb.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [88]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [102]:
def answer_six():
    
    spam_data['length'] = [len(w) for w in spam_data['text']]
    avgnotspam = spam_data.groupby('target')['length'].mean()[0]
    avgspam = spam_data.groupby('target')['length'].mean()[1]
    
    return (avgnotspam, avgspam)

In [103]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [106]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')


### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [133]:
from sklearn.svm import SVC

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)

    l = [len(w) for w in X_train]
    newX = add_feature(X_train_vectorized, l)

    svm = SVC(C=10000).fit(newX, y_train)

    Pred = svm.predict(add_feature(vect.transform(X_test), [len(w) for w in X_test]))
    
    return roc_auc_score(y_test, Pred)

In [134]:
answer_seven()

0.9581366823421557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [137]:
def answer_eight():
    
    spam_data['ndigits'] = spam_data['text'].str.count(r'\d')
    digitnotspam = spam_data.groupby('target')['ndigits'].mean()[0]
    digitspam = spam_data.groupby('target')['ndigits'].mean()[1]
    
    return (digitnotspam, digitspam)

In [138]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [154]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    newX = add_feature(X_train_vectorized, [[len(w) for w in X_train], X_train.str.count(r'\d')])

    lr = LogisticRegression(C=100).fit(newX, y_train)

    X_test_vectorized = vect.transform(X_test)
    newXt = add_feature(X_test_vectorized, [[len(w) for w in X_test], X_test.str.count(r'\d')])

    predictions = lr.predict(newXt)

    return roc_auc_score(y_test, predictions)

#newX.toarray().shape #(4179, 3385)
#X_train_vectorized.toarray().shape #(4179, 3383)

In [155]:
answer_nine()

0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [240]:
def answer_ten():
    
    chardf = pd.DataFrame({'Target': list(spam_data['target']), 'Nonword': list(spam_data['text'].str.count(r'\W'))})
    avgnotspam = chardf.groupby('Target')['Nonword'].mean()[0]
    avgspam = chardf.groupby('Target')['Nonword'].mean()[1]
    
    return (avgnotspam, avgspam)

In [241]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [224]:
def answer_eleven():
    
    vect = CountVectorizer(ngram_range=(2, 5), analyzer='char_wb', min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)

    ##Features##
    length_of_doc = [len(w) for w in X_train]
    digit_count = X_train.str.count(r'\d')
    non_word_char_count = X_train.str.count(r'[\w\W]')

    newX = add_feature(X_train_vectorized, [length_of_doc, digit_count, non_word_char_count])

    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression(C=100).fit(newX, y_train)

    X_test_vectorized = vect.transform(X_test)

    length_of_doc = [len(w) for w in X_test]
    digit_count = X_test.str.count(r'\d')
    non_word_char_count = X_test.str.count(r'[\w\W]')

    newXt = add_feature(X_test_vectorized, [length_of_doc, digit_count, non_word_char_count])
    predictions = lr.predict(newXt)
    auc_score = roc_auc_score(y_test, predictions)

    coef = list(lr.coef_[0])
    coef.sort()
    smallest = coef[:10]
    largest = coef[:-11:-1]
    
    return (auc_score, smallest, largest)
#CountVectorizer?

In [225]:
answer_eleven()

(0.9788593110707434,
 [-0.8313582979501098,
  -0.7989184891799098,
  -0.6624724814830801,
  -0.6543184778422814,
  -0.6312604944371589,
  -0.5886281796382773,
  -0.534070540772252,
  -0.5081448013607297,
  -0.5077633412855984,
  -0.4996064315522647],
 [1.264566490605265,
  0.6078113270193584,
  0.5493060464895717,
  0.5433364586514828,
  0.5374935175908957,
  0.5279541352580774,
  0.5258405370547764,
  0.5250765378746814,
  0.5241228132753276,
  0.5222148930805526])